In [1]:
import tensorflow as tf
from utils import load_data, split_data
import os

print(f"tensorflow: {tf.__version__}")
print(f"keras: {tf.keras.__version__}")

tensorflow: 2.0.0-alpha0
keras: 2.2.4-tf


In [5]:
%env DATA_DIR ../data/GSE92742_Broad_LINCS

data_dir = os.environ['DATA_DIR']

data_fname = 'GSE92742_Broad_LINCS_Level4_ZSPCINF_mlr12k_n1319138x12328.gctx'
data_path = os.path.join(data_dir, data_fname)

sample_meta_fname = 'GSE92742_Broad_LINCS_inst_info.txt'
sample_meta_path = os.path.join(data_dir, sample_meta_fname)

env: DATA_DIR=../data/GSE92742_Broad_LINCS


In [19]:
# Read in raw data, selecting for cells by treatment
pert_types = [
    'trt_cp',       # treated with compound
    'ctl_vehicle',  # control for compound treatment (e.g DMSO) 
    'ctl_untrt'     # untreated samples
]

sample_labs, gene_labs, data = load_data(data_path, sample_meta_path, pert_types)
print(f"data size: {data.shape}")

total data size: (722663, 978)


In [20]:
# Split data into training, validation, and testing
train, val, test = split_data(data, sample_labs, 0.2)
print(f"training size: {train[0].shape[0]}")
print(f"validation size: {val[0].shape[0]}")
print(f"testing size: {test[0].shape[0]}")

training size: 462504
validation size: 115626
test size: 144533
